# Imports

In [12]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import os

# Load Dataset

In [33]:
dataset1 = pd.read_csv('GooglePlay.csv')
reference_dataset = pd.read_csv('Playstore_final.csv')
# remove columns named 'Unnamed' in df1
columns_to_drop = [col for col in dataset1.columns if 'Unnamed' in col]
dataset1 = dataset1.drop(columns=columns_to_drop)
reference_dataset = reference_dataset.loc[:, ~reference_dataset.columns.str.contains('^Unnamed')]

C:\Users\fathi\AppData\Local\Temp\ipykernel_4384\3838032470.py:2: DtypeWarning: Columns (7,12,29,30,33,35,37,38,40,42,43,44,46,47,48,49,50,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  reference_dataset = pd.read_csv('Playstore_final.csv')


### Find columns with the same names

In [34]:
columns_to_check = reference_dataset.columns
columns_to_check1 = dataset1.columns
same_columns = [col for col in columns_to_check if col in columns_to_check1]
dif_col_in_googleplay = [col for col in columns_to_check if col not in columns_to_check1]
dif_col_in_playstore = [col for col in columns_to_check1 if col not in columns_to_check]
print('#-- same cols:', same_columns)
print('#-- googleplay dif cols:', dif_col_in_googleplay)
print('#-- palystore dif cols:', dif_col_in_playstore)

#-- same cols: ['Category', 'Rating', 'Installs', 'Price', 'Size', 'Content Rating', 'Reviews']
#-- googleplay dif cols: ['App Name', 'App Id', 'Rating Count', 'Minimum Installs', 'Free', 'Currency', 'Minimum Android', 'Developer Id', 'Developer Website', 'Developer Email', 'Released', 'Last update', 'Privacy Policy', 'Ad Supported', 'In app purchases', 'Editor Choice', 'Summary', 'Android version Text', 'Developer', 'Developer Address', 'Developer Internal ID', 'Version']
#-- palystore dif cols: ['App', 'Type', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [35]:
# Rename the 'App Name' column in the reference dataset to 'App' to match dataset1
# reference_dataset.rename(columns={'App Name': 'App'}, inplace=True)

# Optionally, clean the 'App' columns if they have leading/trailing spaces or inconsistencies
# dataset1['App'] = dataset1['App'].str.strip()
# reference_dataset['App'] = reference_dataset['App'].str.strip()

# Merge Dataset

In [36]:
merged_data = pd.merge(dataset1, reference_dataset, how='inner', left_on="App", right_on="App Name")

In [37]:
# Examine the first few rows of the merged data
print(merged_data.head())
print(len(merged_data))

                                       App      Category_x  Rating_x  \
0  Smoke Effect Photo Maker - Smoke Editor  ART_AND_DESIGN       3.8   
1                         Infinite Painter  ART_AND_DESIGN       4.1   
2                     Garden Coloring Book  ART_AND_DESIGN       4.4   
3           Tattoo Name On My Photo Editor  ART_AND_DESIGN       4.2   
4           Tattoo Name On My Photo Editor  ART_AND_DESIGN       4.2   

  Reviews_x Size_x   Installs_x  Type Price_x Content Rating_x        Genres  \
0       178    19M      50,000+  Free       0         Everyone  Art & Design   
1     36815    29M   1,000,000+  Free       0         Everyone  Art & Design   
2     13791    33M   1,000,000+  Free       0         Everyone  Art & Design   
3     44829    20M  10,000,000+  Free       0             Teen  Art & Design   
4     44829    20M  10,000,000+  Free       0             Teen  Art & Design   

   ... Ad Supported In app purchases Editor Choice  \
0  ...         True            F

## Merge Category Columns

### Analyse Category

In [38]:
# compare values of category column in both datasets

# unique categories in reference dataset
category_1 = sorted(pd.Series(reference_dataset['Category'].unique()).map(lambda x: str(x).lower().replace('_', ' ')))
print(f'#-- category 1 (df):\n {category_1}\n#-- counts: {len(category_1)}\n')

# unique categories in df1
category_2 = (sorted(pd.Series(dataset1['Category'].unique()).map(lambda x: str(x).lower().replace('&', 'and'))))
print(f'#-- category 2 (df1):\n {category_2}\n#-- counts: {len(category_2)}\n')

# find common categories
common_categories = [cate for cate in category_2 if cate in category_1]
print(f'#-- common categories:\n {common_categories}\n#-- counts: {len(common_categories)}\n')

# find different categories in df
diff_categories_1 = [cate for cate in category_1 if cate not in category_2]
print(f'#-- different categories 1:\n {diff_categories_1}\n#-- counts: {len(diff_categories_1)}\n')

# find different categories in df1
diff_categories_2 = [cate for cate in category_2 if cate not in category_1]
print(f'#-- different categories 2:\n {diff_categories_2}\n#-- counts: {len(diff_categories_2)}')

#-- category 1 (df):
 ['action', 'adventure', 'arcade', 'art & design', 'auto & vehicles', 'beauty', 'board', 'books & reference', 'business', 'card', 'casino', 'casual', 'comics', 'communication', 'dating', 'education', 'educational', 'entertainment', 'events', 'finance', 'food & drink', 'health & fitness', 'house & home', 'libraries & demo', 'lifestyle', 'maps & navigation', 'medical', 'music', 'music & audio', 'nan', 'news & magazines', 'parenting', 'personalization', 'photography', 'productivity', 'puzzle', 'racing', 'role playing', 'shopping', 'simulation', 'social', 'sports', 'strategy', 'tools', 'travel & local', 'trivia', 'video players & editors', 'weather', 'word']
#-- counts: 49

#-- category 2 (df1):
 ['1.9', 'art_and_design', 'auto_and_vehicles', 'beauty', 'books_and_reference', 'business', 'comics', 'communication', 'dating', 'education', 'entertainment', 'events', 'family', 'finance', 'food_and_drink', 'game', 'health_and_fitness', 'house_and_home', 'libraries_and_demo',

### Find Inconstintency

In [39]:
count_inconsistency_category = 0
for i in range(len(merged_data['Category_x'])):
    category_x = merged_data['Category_x'][i].lower().replace('_', ' ') 
    category_y = merged_data['Category_y'][i].lower().replace('&', 'and')
    if  category_x != category_y:
        print(f"{i} > {category_x}: {category_y}")
        count_inconsistency_category += 1

count_inconsistency_category

6 > art and design: entertainment
7 > art and design: casual
8 > art and design: entertainment
9 > art and design: entertainment
10 > art and design: casual
12 > art and design: entertainment
18 > art and design: books and reference
20 > family: art and design
22 > art and design: house and home
27 > auto and vehicles: maps and navigation
32 > auto and vehicles: shopping
37 > beauty: photography
38 > beauty: photography
40 > beauty: lifestyle
50 > books and reference: education
55 > books and reference: education
56 > books and reference: tools
57 > books and reference: education
58 > books and reference: education
59 > books and reference: education
60 > books and reference: education
61 > books and reference: education
63 > books and reference: education
64 > books and reference: education
68 > books and reference: education
71 > books and reference: education
72 > books and reference: education
73 > books and reference: education
74 > books and reference: education
75 > books and re

1138 > lifestyle: house and home
1151 > lifestyle: house and home
1153 > lifestyle: health and fitness
1154 > lifestyle: music and audio
1155 > lifestyle: music and audio
1158 > lifestyle: parenting
1159 > game: arcade
1160 > game: arcade
1161 > game: arcade
1162 > game: arcade
1163 > game: arcade
1164 > game: arcade
1165 > game: casual
1166 > game: casual
1167 > game: casual
1168 > game: casual
1169 > game: casual
1170 > game: casual
1171 > family: casual
1172 > game: card
1173 > game: card
1174 > game: card
1175 > game: card
1176 > game: card
1177 > game: board
1178 > game: card
1179 > game: card
1180 > game: card
1181 > game: card
1182 > game: card
1183 > game: card
1184 > game: card
1185 > game: card
1186 > game: board
1187 > game: card
1188 > game: card
1189 > game: card
1190 > game: card
1191 > game: card
1192 > game: card
1193 > game: card
1194 > game: card
1195 > game: card
1196 > game: card
1197 > game: card
1198 > game: card
1199 > game: card
1200 > game: card
1201 > game: ca

2048 > game: word
2049 > game: word
2050 > game: word
2051 > game: word
2052 > game: word
2053 > game: word
2054 > game: puzzle
2055 > game: word
2056 > game: word
2057 > game: puzzle
2058 > game: word
2059 > game: word
2060 > game: puzzle
2061 > game: puzzle
2062 > game: puzzle
2063 > game: word
2064 > game: word
2065 > game: puzzle
2066 > game: word
2067 > game: puzzle
2068 > game: word
2069 > game: word
2070 > game: puzzle
2071 > game: puzzle
2072 > game: puzzle
2073 > game: word
2074 > game: word
2075 > game: word
2076 > game: word
2077 > game: puzzle
2078 > game: word
2079 > game: word
2080 > game: word
2081 > game: word
2082 > game: arcade
2083 > game: arcade
2084 > game: arcade
2085 > game: arcade
2086 > game: arcade
2087 > game: puzzle
2088 > game: puzzle
2089 > game: puzzle
2090 > game: puzzle
2091 > family: puzzle
2092 > game: racing
2093 > game: puzzle
2094 > game: puzzle
2095 > game: puzzle
2096 > game: arcade
2097 > game: arcade
2098 > game: arcade
2099 > game: arcade
2100

2452 > family: board
2453 > family: puzzle
2454 > family: board
2455 > family: board
2456 > family: board
2457 > game: board
2458 > game: puzzle
2459 > game: board
2460 > game: board
2461 > game: board
2462 > family: board
2463 > family: puzzle
2464 > family: board
2465 > family: board
2466 > family: puzzle
2467 > family: puzzle
2468 > family: puzzle
2469 > family: board
2470 > family: board
2471 > family: board
2472 > family: board
2473 > family: board
2474 > family: board
2475 > family: strategy
2476 > family: board
2477 > family: puzzle
2478 > family: puzzle
2479 > family: board
2480 > family: strategy
2481 > family: casual
2482 > family: puzzle
2483 > family: puzzle
2484 > family: puzzle
2485 > family: puzzle
2486 > family: puzzle
2487 > family: board
2488 > family: puzzle
2489 > family: board
2490 > family: board
2491 > family: board
2492 > family: board
2493 > family: board
2494 > family: puzzle
2495 > family: board
2496 > family: puzzle
2497 > family: board
2498 > family: casual

3923 > video players: video players and editors
3924 > video players: video players and editors
3925 > video players: video players and editors
3926 > video players: tools
3927 > video players: entertainment
3928 > video players: video players and editors
3929 > video players: video players and editors
3930 > video players: video players and editors
3931 > video players: video players and editors
3932 > video players: video players and editors
3933 > video players: video players and editors
3934 > video players: video players and editors
3935 > video players: tools
3936 > video players: entertainment
3937 > video players: video players and editors
3938 > video players: video players and editors
3939 > video players: video players and editors
3940 > video players: video players and editors
3941 > video players: video players and editors
3942 > video players: video players and editors
3943 > video players: video players and editors
3944 > video players: video players and editors
3945 > v

4678 > game: adventure
4682 > family: strategy
4684 > events: sports
4685 > game: action
4686 > family: adventure
4687 > family: role playing
4688 > family: simulation
4689 > game: action
4690 > family: strategy
4692 > game: action
4693 > family: simulation
4696 > family: casual
4698 > family: puzzle
4699 > family: simulation
4700 > family: puzzle
4702 > family: education
4706 > family: education
4707 > family: education
4708 > family: education
4709 > family: education
4710 > family: education
4711 > family: education
4714 > family: education
4715 > family: education
4717 > family: education
4718 > family: casual
4719 > family: puzzle
4725 > family: role playing
4726 > family: role playing
4727 > family: role playing
4728 > family: role playing
4729 > family: role playing
4730 > family: strategy
4731 > game: action
4732 > family: casual
4733 > family: simulation
4734 > game: action
4735 > game: action
4738 > family: entertainment
4739 > family: entertainment
4740 > family: education
4

5595 > game: arcade
5598 > family: entertainment
5599 > family: entertainment
5600 > family: entertainment
5601 > family: entertainment
5602 > family: entertainment
5607 > game: adventure
5608 > family: role playing
5609 > family: role playing
5610 > family: role playing
5611 > family: role playing
5612 > family: role playing
5614 > family: role playing
5616 > family: role playing
5617 > family: role playing
5618 > game: adventure
5620 > game: action
5623 > game: racing
5624 > game: racing
5626 > family: casual
5627 > family: education
5628 > game: racing
5629 > family: role playing
5630 > game: arcade
5631 > game: board
5632 > family: puzzle
5634 > family: puzzle
5635 > game: board
5636 > family: puzzle
5637 > family: casual
5638 > game: board
5639 > family: casual
5640 > family: entertainment
5641 > health and fitness: food and drink
5642 > family: books and reference
5643 > family: role playing
5644 > family: puzzle
5645 > family: educational
5646 > game: board
5647 > game: action
5

3090

### Fill Not Available Values

In [40]:
merged_data['Category_y'] = merged_data['Category_y'].fillna(merged_data['Category_x'])

### Merge Category columns

In [41]:
category_merged = []

for i in range(len(merged_data['Category_x'])):
    # Fill with the source value
    category_merged.append(merged_data['Category_y'][i])


merged_data = merged_data.drop(columns=['Category_x', 'Category_y'])
merged_data['Category'] = pd.Series(category_merged)
merged_data['Category']

0            Art & Design
1            Art & Design
2            Art & Design
3            Art & Design
4            Art & Design
              ...        
6245                 Card
6246    Books & Reference
6247                Tools
6248               Social
6249            Education
Name: Category, Length: 6250, dtype: object

## Merge Rating Columns

### Find inconstintecy

In [42]:
inconsistent_rates = [
    (rate_x, round(rate_y, 1)) 
    for rate_x, rate_y in 
    zip(merged_data['Rating_x'], merged_data['Rating_y']) if rate_x != round(rate_y, 1)
]
# print(inconsistent_rates)
print(f'count inconsistent rates: {len(inconsistent_rates)}')

count inconsistent rates: 5419


### Merge

In [43]:
handled_rating = []

for index, row in merged_data.iterrows():
    rate_x = row['Rating_x']
    rate_y = round(row['Rating_y'], 1)
    
    if pd.notnull(rate_x) and pd.notnull(rate_y):
        if rate_x != round(rate_y, 1):
            handled_rating.append(np.mean([rate_x, rate_y]))
        else:
            handled_rating.append(rate_x)
    elif pd.isnull(rate_x):
        handled_rating.append(rate_y)
    elif pd.isnull(rate_y):
        non_null_y_ratings = merged_data.loc[merged_data['Rating_y'].notnull(), 'Rating_y']
        handled_rating.append(np.mean(non_null_y_ratings))

merged_data['Rating'] = pd.Series(handled_rating)
merged_data = merged_data.drop(columns=['Rating_x', 'Rating_y'])


In [45]:
# Save the merged data to a new CSV file
merged_data.to_csv('merged_data.csv', index=False)
print(len(merged_data))

6250
